# Extract, Transform, and Load Bank Data
The project uses the ETL process to create a list of the top top banks in the world based on market capital. 

### Install the Required Libraries
The libraries used in this project are:
- pandas: to use a dataframe for easy transformation
- requests: to get the data from the web
- bs4: to scrape the required data
- sqlite3: to create and query a database
- numpy
- datetime: for logging purposes

In [18]:
import pandas as pd 
import sqlite3
import requests
import numpy as np
from datetime import datetime
from bs4 import BeautifulSoup

url = 'https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks'
db_name = 'Banks.db'
table_name = 'Largest_banks'
output_file = 'largest_banks_data.csv'
table_attributes = ['Name', 'MC_USD_Billion']

## Log Message
Keep track of each operation in the ETL process. The following log function provides a timestamp of each operation.

In [19]:
# file to store the logs
log_file = 'code_log.txt'

# Log message function
def log_progress(message):
    timestamp_format = '%Y-%h-%d-%H:%M:%S'
    now = datetime.now()

    # Convert time to a string
    timestamp = now.strftime(timestamp_format)

    # Log the message to the file
    with open(log_file, 'a') as file_log:
        file_log.write(f'{timestamp} : {message}\n')

# Declare the completion of the set up
log_progress('Preliminaries complete. Initiating ETL process')

## The Extraction Stage
In this stage the data is extracted from the web and loaded into a dataframe.

In [20]:
# Extraction function
def extraction(url, table_attr):
    data_frame = pd.DataFrame(columns=table_attr)
    
    try:
        # Load the web page as an HTML document
        response = requests.get(url)
        if response.status_code == 200:
            # Parse the HTML document
            html_content = response.text
            soup = BeautifulSoup(html_content, 'html.parser')

            # Locate the specifeid table after the text: By market Capitalization
            span = soup.find('span', id='By_market_capitalization')
            h2_tag = span.find_parent('h2')
            table = h2_tag.find_next('table')

            # Find the rows of the table
            rows = table.find_all('tr')
            
            # Iterate the rows to extract the bank name and the associated market capital
            for row in rows:
                columns = row.find_all('td')
                if len(columns) > 0:
                    name = columns[1].get_text(strip=True)
                    market_capital = columns[2].get_text(strip=True)

                    # Place data in a dictionary and append it to the DF
                    bank_dict = {table_attr[0]: name, table_attr[1]: float(market_capital)}
                    temp_df = pd.DataFrame(bank_dict, index=[0])
                    data_frame = pd.concat([data_frame, temp_df], ignore_index=True)
            return data_frame     
    except Exception as e:
        print(f'Error loading the page: {e}')
        return None   
            

bank_data_frame = extraction(url, table_attributes)
# Log the extraction phase completion
log_progress('Data extraction complete. Initiating Transformation process')
print(bank_data_frame)

                                      Name  MC_USD_Billion
0                           JPMorgan Chase          432.92
1                          Bank of America          231.52
2  Industrial and Commercial Bank of China          194.56
3               Agricultural Bank of China          160.68
4                                HDFC Bank          157.91
5                              Wells Fargo          155.87
6                        HSBC Holdings PLC          148.90
7                           Morgan Stanley          140.83
8                  China Construction Bank          139.82
9                            Bank of China          136.81


/var/folders/64/w7ll3frd5ll8jdd45h0041pw0000gn/T/ipykernel_4178/1123685148.py:31: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  data_frame = pd.concat([data_frame, temp_df], ignore_index=True)


## The Transform Stage
The data in the dataframe from the previous step is transformed in this stage to meet specific requirements:
- Add 3 new columns to sure 3 more currencies

In [24]:
# Transformation function
def transformation(data, csv_path):
    # Convert the contents of the csv file to a dictionary
    exchange_rate_df = pd.read_csv(csv_path)
    exchange_rate_dict = exchange_rate_df.set_index('Currency').to_dict()['Rate']
    
    # Add 3 columns to the data frame
    data['MC_GBP_Billion'] = [
        np.round(value * exchange_rate_dict['GBP'], 2) for value in data['MC_USD_Billion']]
    data['MC_EUR_Billion'] = [
        np.round(value * exchange_rate_dict['EUR'], 2) for value in data['MC_USD_Billion']]
    data['MC_INR_Billion'] = [
        np.round(value * exchange_rate_dict['INR'], 2) for value in data['MC_USD_Billion']]
    return data

cleaned_data = transformation(bank_data_frame, 'exchange_rate.csv')
# Log the transformation phase completion
log_progress('Data transformation complete. Initiating Loading process')
print(cleaned_data)

print(cleaned_data['MC_EUR_Billion'][4])

                                      Name  MC_USD_Billion  MC_GBP_Billion  \
0                           JPMorgan Chase          432.92          346.34   
1                          Bank of America          231.52          185.22   
2  Industrial and Commercial Bank of China          194.56          155.65   
3               Agricultural Bank of China          160.68          128.54   
4                                HDFC Bank          157.91          126.33   
5                              Wells Fargo          155.87          124.70   
6                        HSBC Holdings PLC          148.90          119.12   
7                           Morgan Stanley          140.83          112.66   
8                  China Construction Bank          139.82          111.86   
9                            Bank of China          136.81          109.45   

   MC_EUR_Billion  MC_INR_Billion  
0          402.62        35910.71  
1          215.31        19204.58  
2          180.94        16138.75

## The Load Stage
The data will loaded loaded to a csv file as well as a database for storage. This will allows other users to further explore the data.

In [22]:
# Load data to a csv file
def load_data_to_csv(data, output_path):
    data.to_csv(output_path, index=False)

stored_file = load_data_to_csv(cleaned_data, output_file)
# Log the completion of the data being loaded to a csv file
log_progress('Data saved to CSV file')

# Load data to a sqlite database
def load_data_to_sqlite(data, sql_con, table_name):
    
    # Add the table to the database
    data.to_sql(table_name, sql_con, if_exists='replace', index=False)
    
# Log initiated sqlite connection
sql_connection = sqlite3.connect(db_name)
log_progress('SQL Connection initiated')
bank_sql_db = load_data_to_sqlite(cleaned_data, sql_connection, table_name)

# Log the completion of the data being loaded to a sqlite database
log_progress('Data loaded to Database as a table, Executing queries')

## Query the Database
Run queries to ensure that the data was stored properly.

In [23]:
def run_query(query, sql_connection):
    query_result = pd.read_sql(query, sql_connection)
    return query_result


# Make 3 query calls
query_all_contents = 'SELECT * FROM Largest_banks'
result_all_contents = run_query(query_all_contents, sql_connection)
print(f'Full Content:\n {result_all_contents}\n')

# Print average MC_USD_Billion
avg_billion = 'SELECT AVG(MC_GBP_Billion) FROM Largest_banks'
result_avg_billion = run_query(avg_billion, sql_connection)
print(f'Average Market Capital: {result_avg_billion}\n')

# Print the top 5 banks
query_top_5 = 'SELECT Name from Largest_banks LIMIT 5'
result_top_5 = run_query(query_top_5, sql_connection)
print(f'Top 5 Banks: {result_top_5}\n')

# Log the completion of the query execution
log_progress('Process Complete')

# Log close of the connection
sql_connection.close()
log_progress('Server Connection closed')

Full Content:
                                       Name  MC_USD_Billion  MC_GBP_Billion  \
0                           JPMorgan Chase          432.92          346.34   
1                          Bank of America          231.52          185.22   
2  Industrial and Commercial Bank of China          194.56          155.65   
3               Agricultural Bank of China          160.68          128.54   
4                                HDFC Bank          157.91          126.33   
5                              Wells Fargo          155.87          124.70   
6                        HSBC Holdings PLC          148.90          119.12   
7                           Morgan Stanley          140.83          112.66   
8                  China Construction Bank          139.82          111.86   
9                            Bank of China          136.81          109.45   

   MC_EUR_Billion  MC_INR_Billion  
0          402.62        35910.71  
1          215.31        19204.58  
2          180.94 